# Optosynth

... is it _really_ synthetic?! :-)

In [1]:
%matplotlib inline

import os
import sys
notebook_path = os.path.abspath('')
sources_path = os.path.abspath(os.path.join(notebook_path, 'sources'))
sys.path.insert(0, sources_path)

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from pprint import pprint

from specs import *
from optosynth import Optosynth

device = torch.device("cpu")
dtype = torch.float32

## Paths

In [2]:
optosynth_data_path = '/home/jupyter/mb-ml-data-disk/Optosynth'
output_path = '/home/jupyter/mb-ml-data-disk/Optosynth_output'
output_prefix = 'optosynth_first_wave'

if not os.path.exists(output_path):
    os.mkdir(output_path)

## (Optional) Download pre-processed Allen Brain data

In [3]:
# if not os.path.exists(optosynth_data_path):
#     os.mkdir(optosynth_data_path)
# assert os.path.exists(optosynth_data_path)
# data_bucket = 'gs://mb_optosynth_data'

In [4]:
# %%bash -s "$data_bucket" "$optosynth_data_path"
# mkdir -p $2
# gsutil -m cp -r $1/* $2

## Load summary of available sweeps

In [5]:
# summary of sweeps
ephys_summary_df = pd.read_csv(
    os.path.join(optosynth_data_path, 'processed_electrophysiology_summary.csv'), index_col=0)
ephys_summary_df.head(5)

,cell_id,sweep_index,stim_amp,n_spikes
0,480116737,0,-110.000000,0
1,480116737,1,-70.000000,0
2,480116737,2,-50.000000,0
3,480116737,3,-30.000002,0
4,480116737,4,10.000000,0


## Optosynth Specs

In [6]:
n_repeat = 5
n_neurons_list = [5, 10, 20]
photon_per_fluorophore_list = [5, 10, 50, 200]

for i_repeat in range(1, n_repeat + 1):

    for n_neurons in n_neurons_list:

        opto_specs = OptosynthSpecs(
            width=512,
            height=180,
            sampling_rate=500,
            duration_per_segment=2.00,
            scale_factor=1.00,
            n_neurons=n_neurons,
            min_neuron_fluorescence_scale_factor=0.1,
            max_neuron_fluorescence_scale_factor=1.0,
            stim_amp_range_list=[
                (25, 50),
                (50, 75),
                (75, 100),
                (100, 125),
                (125, 150),
                (150, 175),
                (175, 200)])

        neuron_specs = SyntheticNeuronSpecs(
            dendritic_backprop_velocity=1e4,
            dendritic_backprop_decay_lengthscale=20,
            min_reporter_density=1,
            max_reporter_density=10,
            reporter_density_var_lengthscale=2,
            ephys_lowpass_freq=250)

        bg_specs = BackgroundFluorescenceSpecs(
            dynamic_n_components=20,
            dynamic_x_lengthscale=10,
            dynamic_y_lengthscale=100,
            dynamic_fluorophore_density_scale=0.5,
            dynamic_temporal_frequency=100,
            static_x_lengthscale=5,
            static_y_lengthscale=5,
            static_min_total_fluorophore_density=0.0,
            static_max_total_fluorophore_density=0.1)    

        v2f_specs = VoltageToFluorescenceSpecs(
            beta=0.01,
            v1=-100,
            f1=0.4,
            v2=50,
            f2=1.0)

        synth = Optosynth(
            opto_specs=opto_specs,
            neuron_specs=neuron_specs,
            bg_specs=bg_specs,
            v2f_specs=v2f_specs,
            cam_specs=None,
            ephys_summary_df=ephys_summary_df,
            optosynth_data_path=optosynth_data_path,
            device=device,
            dtype=dtype)
        
        # generate!
        synth.generate_fluorescence_data()
        
        for photon_per_fluorophore in photon_per_fluorophore_list:
            
            # generate clean and noisy camera data
            cam_specs = CameraSpecs(
                dc_offset=500,
                gaussian_noise_std=10.,
                psf_lengthscale=0.25,
                readout_per_photon=2.2,
                photon_per_fluorophore=float(photon_per_fluorophore))
            synth.reset_camera(cam_specs)
            synth.generate_camera_data()
            
            # save raw data
            full_output_path = os.path.join(
                output_path,
                f"{output_prefix}__{i_repeat}__{n_neurons}__{photon_per_fluorophore}")
            
            synth.save(full_output_path)
            
            # save avi demo
            for movie_type in {'clean', 'noisy'}:
                synth.generate_avi_from_segment(
                    segment_index=(synth.n_segments - 1),
                    movie_type=movie_type,
                    output_root=full_output_path,
                    n_frame_mean_subtraction=100)

Randomly choosing 5 from 446 usable neurons ...
Loading morphology and electrophysiology data ...


  0%|          | 0/5 [00:00<?, ?it/s]

Instantiating the background pattern generator ...
Instantiating the camera ...
Generating ground truth masks ...
Generating synthetic data for segment 1 / 1 ...


  0%|          | 0/1000 [00:00<?, ?it/s]

Generating clean and noisy camera readout for segment 1 / 1 ...


...

Instantiating the background pattern generator ...
Instantiating the camera ...
Generating ground truth masks ...
Generating synthetic data for segment 1 / 1 ...


  0%|          | 0/1000 [00:00<?, ?it/s]

Generating clean and noisy camera readout for segment 1 / 1 ...


  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 